In [4]:

# 3. 准备数据
import pandas as pd
df = pd.read_csv("./data/HK_ali_09988_kline_5min_all.csv")
df['timestamps'] = pd.to_datetime(df['timestamps'])


In [6]:
df.set_index('timestamps', inplace=True)


In [7]:
df

,open,close,high,low,volume,amount
timestamps,,,,,,
2019-11-26 09:35:00,182.45215,184.45215,184.95215,182.45215,15136000,0
2019-11-26 09:40:00,184.35215,183.85215,184.55215,183.45215,4433300,0
2019-11-26 09:45:00,183.85215,183.35215,183.95215,182.95215,3070900,0
2019-11-26 09:50:00,183.35215,183.75215,183.95215,183.25215,2288600,0
2019-11-26 09:55:00,183.85215,183.85215,184.15215,183.75215,1770000,0
...,...,...,...,...,...,...
2025-09-17 15:40:00,161.90000,161.90000,162.10000,161.90000,1215300,0
2025-09-17 15:45:00,162.00000,161.80000,162.00000,161.80000,1154100,0
2025-09-17 15:50:00,161.90000,161.60000,161.90000,161.60000,1554600,0


In [33]:
import pandas as pd

class PineSeries:
    """
    PineSeries 是对 pd.Series 的轻量级包装，支持：
    1. 直接访问变量返回最新一条数据（close 等价于 close.iloc[-1]）
    2. 通过索引访问历史数据（close[1] 返回前一天的数据）
    3. 支持基础运算，运算作用于整个 Series（如 close*2 表示每个元素乘以 2）
    """
    def __init__(self, data: pd.Series):
        # 确保索引为时间序列且升序
        if not isinstance(data.index, pd.DatetimeIndex):
            raise TypeError("PineSeries 仅支持 DatetimeIndex 的 Series")
        self._data = data
        
    
    # 支持 close[1] 形式访问历史数据
    def __getitem__(self, key):
        if isinstance(key, int):
            # 正整数索引：1 表示前一天，2 表示前两天，以此类推
            # 负整数索引：-1 表示最新，-2 表示前一天，以此类推
            if key > 0:
                return self._data.iloc[-(key + 1)]
            elif key < 0:
                return self._data.iloc[key]
            else:
                return self._data.iloc[-1]
        return self._data[key]

    # 支持基础运算，返回新的 PineSeries
    def __mul__(self, other):
        return PineSeries(self._data * other)

    def __rmul__(self, other):
        return PineSeries(other * self._data)

    def __add__(self, other):
        return PineSeries(self._data + other)

    def __radd__(self, other):
        return PineSeries(other + self._data)

    def __sub__(self, other):
        return PineSeries(self._data - other)

    def __rsub__(self, other):
        return PineSeries(other - self._data)

    def __truediv__(self, other):
        return PineSeries(self._data / other)

    def __rtruediv__(self, other):
        return PineSeries(other / self._data)

    def __pow__(self, other):
        return PineSeries(self._data ** other)

    def __rpow__(self, other):
        return PineSeries(other ** self._data)

    # 保留原始 Series 的属性和方法
    @property
    def series(self):
        return self._data

    def to_series(self):
        return self._data
    
    def __call__(self):
        return self._data.iloc[-1]


In [34]:
close=PineSeries(df['close'])

In [48]:
df

,timestamps,open,close,high,low,volume,amount
0,2019-11-26 09:35:00,182.45215,184.45215,184.95215,182.45215,15136000,0
1,2019-11-26 09:40:00,184.35215,183.85215,184.55215,183.45215,4433300,0
2,2019-11-26 09:45:00,183.85215,183.35215,183.95215,182.95215,3070900,0
3,2019-11-26 09:50:00,183.35215,183.75215,183.95215,183.25215,2288600,0
4,2019-11-26 09:55:00,183.85215,183.85215,184.15215,183.75215,1770000,0
...,...,...,...,...,...,...,...
93907,2025-09-17 15:40:00,161.90000,161.90000,162.10000,161.90000,1215300,0
93908,2025-09-17 15:45:00,162.00000,161.80000,162.00000,161.80000,1154100,0
93909,2025-09-17 15:50:00,161.90000,161.60000,161.90000,161.60000,1554600,0
93910,2025-09-17 15:55:00,161.70000,161.90000,162.00000,161.60000,2015300,0


161.6

In [63]:
a=pd.Series([df.iloc[400+1:400+2]["timestamps"]],name='timestamps')
b=pd.Series([df.iloc[400+1]["timestamps"]],name='timestamps')


In [75]:
df.iloc[-1:3]

,timestamps,open,close,high,low,volume,amount
